# 0. Preparando o ambiente

In [1]:
!pip install pandas scikit-learn numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from sklearn.neighbors import KNeighborsClassifier

# 1. Baixando o conjunto de dados de outra forma

In [5]:
!wget https://archive.ics.uci.edu/static/public/53/iris.zip

--2024-02-27 15:13:42--  https://archive.ics.uci.edu/static/public/53/iris.zip
Resolvendo archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Conectando-se a archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: não especificada
Salvando em: ‘iris.zip’

iris.zip                [ <=>                ]   3,65K  --.-KB/s    em 0s      

2024-02-27 15:13:43 (527 MB/s) - ‘iris.zip’ salvo [3738]



In [6]:
!unzip iris.zip

Archive:  iris.zip
  inflating: Index                   
  inflating: bezdekIris.data         
  inflating: iris.data               
  inflating: iris.names              


# 2. Utilizando o pandas para realizar a leitura dos dados

In [7]:
ds = pd.read_csv('./iris.data',
      names=["sepal_length", "sepal_width", "petal_length", "petal_width", "especie"])

In [8]:
ds.head()

,sepal_length,sepal_width,petal_length,petal_width,especie
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
ds.sepal_width

0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
145    3.0
146    2.5
147    3.0
148    3.4
149    3.0
Name: sepal_width, Length: 150, dtype: float64

In [10]:
ds.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [11]:
ds.groupby('especie').size()

especie
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
dtype: int64

# 3. Preparação dos dados

## 3.1. Separação dos dados usados para a classificação da própria classe a ser inferida

In [12]:
X = ds.iloc[:, 0:4].values
y = ds.iloc[:, 4].values

In [13]:
y[0:2], y[50:52], y[100:102], len(y)

(array(['Iris-setosa', 'Iris-setosa'], dtype=object),
 array(['Iris-versicolor', 'Iris-versicolor'], dtype=object),
 array(['Iris-virginica', 'Iris-virginica'], dtype=object),
 150)

In [14]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## 3.2. Separando os conjuntos em treino e teste

O tamanho do conjunto de testes foi de 20%.

Nota, geralmente esta separação é feita em treino, validação e testes. Mas este é um assunto para discussão futura.

Além disso, note também que não houve preocupação em relação a homogeneidade da amostra, pois pode ser que nos 20% não haja amostras iguais de cada espécie.

In [15]:
X_train, X_test, y_train, y_test = \
  train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
len(X_train), len(y_train)

(120, 120)

In [17]:
len(X_test), len(y_test)

(30, 30)

# 4. Treinando o modelo

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)

In [19]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

# 5. Verificando o resultado do modelo

In [20]:
y_pred = knn.predict(X_test)
print(y_pred)

['Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa']


In [21]:
y_test

array(['Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa'], dtype=object)

## 5.1. Acurácia

In [22]:
acuracia = accuracy_score(y_test, y_pred) * 100

In [23]:
print('Acurácia ' + str(round(acuracia, 2)) + '%.')

Acurácia 96.67%.


## 5.2. Matriz de confusão

In [24]:
confusion_matrix(y_test, y_pred,
      labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

array([[11,  0,  0],
       [ 0, 12,  1],
       [ 0,  0,  6]])

Segue a forma visual para entender melhor essa matriz de resultado:

\begin{array}{lccc}
          & \text{Predito} \\
    \text{Real} & setosa & versicolor & virginica\\
    setosa &  11 & 0 & 0\\
    versicolor & 0 & 12 & 1 \\
    virginica & 0 & 0 & 6 \\
\end{array}

Observe que os valores vão mudar dependendo do sorteio que foi feito na divisão dos conjuntos de treino e teste.

Note que neste caso o sistema confundiu 1 planta da espécie *Iris versicolor*, "pensando" que era da espécie *Iris virginica*.

Assim, as linhas representam as classes reais e
as colunas representam as classes preditas.